# 背景
我们的目的是在一个方便的环境里，体验一下类似 GPT 这样架构的语言模型的训练过程，如果有更条件使用更接近于真实情况的计算环境和足够的预算，那么我更推荐通过Karpathy的两个知名项目来学习体验这一过程：
- [nanoGPT: 体验小型 GPT 模型的预训练和微调过程](https://github.com/karpathy/nanoGPT)
- [nanochat: 100 美元即可获得一个基本可用的 ChatGPT 产品](https://github.com/karpathy/nanochat)

但对于没有便利的算力条件或预算的情况，我们构建了这个项目，简化问题规模、缩小数据集和模型架构，在有限的环境里“照虎画猫”，模仿 nanochat 的训练过程，完成一个专用的小模型训练。

项目设定的主题是“**训练一个能够模仿某位诗人、基于一定的诗词格式来生成一首完整诗词的语言模型**”，这个问题足够小但也有一点点深度，同时感谢 [chinese-poetry](https://github.com/chinese-poetry/chinese-poetry) 项目提供了非常好的基础数据，让数据获取非常容易。在chinese-poetry项目的基础上，我们提取并整理了一个小型数据集，集成了唐诗、宋词和诗经，基于这些语料来制作这个小型的语言模型。

> 由于数据集局限性很大，因此这个模型没法用来“对话”，它也不理解除了中文之外的其他语言、数字以及符号。

整个的训练过程仿照 nanochat 项目，被拆解成了 4 个部分：PRE、MID、SFT、RL，每个阶段有不同的训练目标，会使用不同样式的数据以及不同的训练方法来完成。我会用下面的表格，对比着 nanochat 项目来解释每个训练阶段的目的。多数的训练策略都比较相似，除了 RL（这个阶段不是必选的）。

| 过程 | nanopoet                        | nanochat                                       |
|----|---------------------------------|------------------------------------------------|
| PRE | 使用诗词内容拼接的字符串训练，学习诗词的语言规律        | 使用FineWeb 数据集，学习通用语言知识和世界知识                    |
| MID | 引入诗词作者、标题、形式信息以及特殊 Token，学习诗词风格 | 使用混合任务数据（GSM8K、MMLU、SmolTalk 等），增强推理、知识问答和对话能力 |
| SFT | 精选作者、形式的诗词，精调通过元数据生成正确诗词的能力     | 使用 SmolTalk 对话数据，精调对话能力，学习多轮对话交互               |
| RL | 通过 RL 强化模型生成特定作者、风格诗词的质量        | 基于GSM8K数据进行强化学习，增强模型推理能力                       |

---
在这个笔记里，我们会先做一些准备工作，包括加载数据，对数据进行简单分析，设计特殊 Token 并实现一个最简单的分词器。

# 数据
在开始正式的模型构建和训练工作之前，需要先分析一下数据的基本状况，这会帮助我们确认一些模型设计和训练策略的细节。同时会设计一些特殊的分隔符，来帮助我们完成“指定作者和形式生成完整诗词”的目标。

In [14]:
# 加载数据
import json

with open('../raw/base_poetry_data.jsonl') as f:
    data = [json.loads(l) for l in f]

print("数据属性", list(data[0].keys()))
print("诗词数量", len(data))
print("作者数量", len(set([d["author"] for d in data])))
print("形式数量", len(set([d["style"] for d in data])))

# 提取数据中所有的字符种类（数据中只有汉字、中文逗号和中文句号）
chars = sorted(list(set("".join(["".join(list(d.values())) for d in data]))))
print("字符种类", len(chars))
print("前后10种字符", chars[:10] + chars[-10:])

数据属性 ['content', 'title', 'author', 'style']
诗词数量 241440
作者数量 9829
形式数量 631
字符种类 11857
前后10种字符 ['。', '㑂', '㑹', '㑾', '㒇', '㒟', '㒿', '㔩', '㔶', '㕙', '龚', '龛', '龜', '龞', '龟', '龠', '龡', '龢', '龦', '，']


In [24]:
# 选择一些最常见的诗词形式和作者（包括了简体和繁体名字），这些作为我们后续精调的数据范围
styles1 = ["七言律诗", "七言绝句", "五言律诗", "五言绝句"]
styles2 = ["浣溪沙", "水调歌头", "西江月", "鹧鸪天", "沁园春", "蝶恋花"]
authors1 = ["苏轼", "辛弃疾", "李清照", "柳永", "欧阳修", "李白", "杜甫", "白居易", "王维", "李商隐", "陆游", "杨万里", "黄庭坚", "王安石", "朱熹"]
authors2 = ["蘇軾", "辛棄疾", "李清照", "柳永", "歐陽修", "李白", "杜甫", "白居易", "王維", "李商隱", "陸游", "楊萬里", "黃庭堅", "王安石", "朱熹"]
# 由于 "七言律诗", "七言绝句", "五言律诗", "五言绝句" 这四种形态占了数据中的绝大多数，因此制作精选数据时，对这 4 种形态的诗词也使用作者做一个过滤，缩小数据范围
def filter_function(p):
    if p["author"] in authors1 + authors2:
        return True
    if p["style"] in styles2:
        return True
    return False

filtered = list(filter(filter_function, data))
# 并且为了提高数据质量，把繁体的作者名字统一都改成对应的简体
updated = []
for p in filtered:
    if p["author"] not in authors2:
        updated.append(p)
    else:
        new_author = authors1[authors2.index(p["author"])]
        new_p = {}
        new_p.update(p)
        new_p["author"] = new_author
        updated.append(new_p)
print("精选后诗词数量:", len(updated))

精选后诗词数量: 24538 24538


# 分隔符
为了能让一个 GPT 架构风格的模型（Decoder-only）能按照指定的作者、形式等信息输出诗词，我们需要设定一些特殊的标识符，来对数据的格式做一些切分。
就像制作一个 ChatGPT，会使用类似`<|user|>`、`<|assistant|>`这样的标识符一样。
由于我们的数据很干净，只包含了汉字、中文逗号、中文句号，为了简化，我们指定一些**字母**来作为分隔符。

In [28]:
BEGIN = "B"
PADDING = "P"
UNKNOWN = "U"

AUTHOR_START = "A"
AUTHOR_END = "a"

STYLE_START = "S"
STYLE_END = "s"

TITLE_START = "T"
TITLE_END = "t"

CONTENT_START = "C"
CONTENT_END = "c"

def encode_poem(poem):
    return (
        f"{BEGIN}"
        f"{AUTHOR_START}{poem['author']}{AUTHOR_END}"
        f"{STYLE_START}{poem['style']}{STYLE_END}"
        f"{TITLE_START}{poem['title']}{TITLE_END}"
        f"{CONTENT_START}{poem['content']}{CONTENT_END}"
    )

# 基于这样的设计，一首诗词的数据被编码成模型使用的序列化形式就变成了如下的形式，未来模型也会输出这样的序列
# 模型输出从 B 开始，找到输出 C 和 c 之间的内容，就是对应的诗词
print(encode_poem(updated[0]))


BA陆游aS五言律诗sT送仲高兄宫學秩滿赴行在tC兄去游東閤，才堪直北扉。莫憂持槖晚，姑記乞身歸。道義無今古，功名有是非。臨分出苦語，不敢計從違。c


# 分词器 Tokenizer
对于这个项目，我们选用最简单的分词器——字符分词器，每一个字符被看作成一个词元（Token），之前使用英文字母做为特殊分隔符，就是为了方便直接使用字符分词器。当然使用 BPE 算法训练一个简单的分词器也是可以的。

In [39]:
class CharTokenizer:

    def __init__(self, raw_text, special_tokens):
        self.chars = special_tokens + sorted(list(set(raw_text)))
        self.stoi = {ch: i for i, ch in enumerate(self.chars)}
        self.itos = {i: ch for i, ch in enumerate(self.chars)}
        self.encode = lambda s: [self.stoi.get(d, UNKNOWN) for d in s]
        self.decode = lambda l : ''.join([self.itos[i] for i in l])
        self.vocab_size = len(self.chars)

tokenizer = CharTokenizer(
    raw_text="".join(["".join(list(d.values())) for d in data]),
    special_tokens=[
        BEGIN,PADDING,UNKNOWN,AUTHOR_START,AUTHOR_END,STYLE_START,STYLE_END,TITLE_START,TITLE_END,CONTENT_START,CONTENT_END
    ]
)
poem_demo = encode_poem(updated[0])
encoded_poem = tokenizer.encode(poem_demo)
decoded_poem = tokenizer.decode(encoded_poem)
print("诗词序列", poem_demo)
print("编码结果", encoded_poem)
print("解码结果",decoded_poem)
print(poem_demo == decoded_poem)


诗词序列 BA陆游aS五言律诗sT送仲高兄宫學秩滿赴行在tC兄去游東閤，才堪直北扉。莫憂持槖晚，姑記乞身歸。道義無今古，功名有是非。臨分出苦語，不敢計從違。c
编码结果 [0, 3, 10548, 4926, 4, 5, 349, 8876, 2775, 9130, 6, 7, 9779, 408, 11281, 740, 2155, 2120, 6473, 5038, 9405, 8634, 1589, 8, 9, 740, 1137, 4926, 3976, 10438, 11867, 3205, 1714, 6100, 1038, 3201, 11, 7998, 3081, 3316, 4307, 3812, 11867, 1937, 8892, 325, 9576, 4500, 11, 9843, 7404, 5322, 387, 1166, 11867, 971, 1193, 3909, 3784, 10728, 11, 7722, 871, 862, 7885, 8962, 11867, 271, 3652, 8880, 2786, 9845, 11, 10]
解码结果 BA陆游aS五言律诗sT送仲高兄宫學秩滿赴行在tC兄去游東閤，才堪直北扉。莫憂持槖晚，姑記乞身歸。道義無今古，功名有是非。臨分出苦語，不敢計從違。c
True


# 最后的统计准备
为了后续能够合理地确认模型的结构参数以及各个阶段的训练参数，最后我们要基于选用的分词器来给数据做一些统计工作。这里使用的是字符级的分词器，所以其实统计分词器的编码数据和直接统计原数据的区别不大，但如果你决定采用其他的分词器，比如 BPE 算法训练出来的分词器，那么这些统计工作一定要在分词器训练完成后进行，毕竟最终决定模型数量的是分词器的输出，而非原始数据。

In [45]:
# 词表大小
print("Token 种类", tokenizer.vocab_size)

# PRE 阶段，我们打算使用所有诗词内容的直接拼接作为训练数据，不带有任何元信息
pre_data = tokenizer.encode("".join([d["content"] for d in data]))
print("PRE 数据量", len(pre_data))

# MID 阶段，我们打算在诗词的基础上加入元信息以及与它们有关的特殊 token
mid_sequences = [encode_poem(d) for d in data]
mid_data = tokenizer.encode("".join(mid_sequences))
print("MID 数据量", len(mid_data))

# SFT 阶段，数据在形式上与 MID 区别不大，但范围会缩小到精选后的数据内
# RL 阶段的数据是不确定的，暂时不在这里关心
sft_sequences = [encode_poem(d) for d in updated]
sft_data = tokenizer.encode("".join(sft_sequences))
print("SFT 数据量", len(sft_data))

# 另外一个重要的指标是我们要保证模型的上下文大小要大于我们需要生成的诗歌最大情况，并且还有一定冗余
print("MID 数据中最长序列", max([len(d) for d in mid_sequences]))
print("SFT 数据中最长序列", max([len(d) for d in sft_sequences]))


Token 种类 11868
PRE 数据量 11186141
MID 数据量 16853134
SFT 数据量 1806430
MID 数据中最长序列 444
SFT 数据中最长序列 242
